In [1]:
fasta_file = '../data/RSV2-reduced.fasta'
date_regex = '[^s]*$'

In [2]:
import topology_inference

raxml_tree, raxml_params = topology_inference.infer_topology_raxml(fasta_file, 'fasta', 'out', subst_model='HKY85')
raxml_params

{'alpha': 1.042979,
 'rates': {'ac': 1.0,
  'ag': 12.053774,
  'at': 1.0,
  'cg': 1.0,
  'ct': 12.053774,
  'gt': 1.0},
 'frequencies': [0.38778, 0.356966, 0.081073, 0.17418]}

In [3]:
rooted_tree, estimated_rate = topology_inference.root_topology(fasta_file, 'fasta', 'out', date_regex, raxml_tree)
estimated_rate

0.0022298379517620846